In [1]:
import os
import lightgbm
import re

import pandas as pd
import numpy as np
import dask.dataframe as dd

from femr.datasets import PatientDatabase

from src.default_paths import path_root, path_extract
from src.io import read_pkl, read_msgpack
from src.utils import list_dir

In [2]:
path_models = os.path.join(path_root, "data/adapter_models/count_mimic")
path_features = os.path.join(path_root, "data/features/count_mimic")
path_labels = os.path.join(path_root, "data/labels")
path_concept_table = "/hpf/projects/lsung/phi/data/mimic_omop_parquet/concept"

df_concepts = dd.read_parquet(path_concept_table).compute()

In [3]:
topN = 50
tasks = list_dir(path_labels)
for task in tasks:
    model = read_pkl(os.path.join(path_models, task, "model.pkl"))
    model_info = read_pkl(os.path.join(path_models, task, "model_info.pkl"))
    featurizer = read_pkl(os.path.join(path_features, task, "preprocessed_featurizers.pkl"))
    featurized_patients = read_pkl(os.path.join(path_features, task, "featurized_patients.pkl"))
    features = featurized_patients[0]
    labels = featurized_patients[2]

    imp = model.feature_importances_
    ind_topN = np.argpartition(imp, -topN)[-topN:]
    imp_topN = imp[ind_topN]
    imp_rank = (-imp_topN).argsort().argsort()+1

    df_results = pd.DataFrame()

    for i, ind in enumerate(ind_topN): 
        col = featurizer.get_column_name(ind)
        rank = imp_rank[i]
        quantile_bin = np.nan
        imp = imp_topN[i]
        n_pos = (features[:, ind]!=0).sum()
        n_prev = (features[:, ind]!=0).mean()
        mean_pos = features[np.where(labels==True)[0],ind].mean()
        mean_neg = features[np.where(labels==False)[0],ind].mean()
        in_sk_dict = 0
        in_stanford_dict = 0
        
        if "AgeFeaturizer" in col:
            concept_vocabulary = np.nan,
            concept_code = np.nan,
            time_bin = np.nan,
            concept_name = "Age",
            concept_domain = np.nan,
            
        else:
            concept_vocabulary = col.split("),")[1].split("/")[0]
            concept_code = col.split("),")[1].split("/")[1].split("_")[0]
            time_bin = col.split("),")[1].split("/")[1].split("_")[1].split(",")[0]
            re_res = re.search(r'(?<=\[)(.*?)(?=\))', col)
            

            if re_res:
                quantile_bin = re_res.groups()[0]

            if "[" in concept_code:
                concept_code = concept_code.split(" ")[0]

            concept_name = df_concepts.query("concept_code==@concept_code")["concept_name"].values[0]
            concept_domain = df_concepts.query("concept_code==@concept_code")["domain_id"].values[0]
            
        df_results = pd.concat((
            df_results,
            pd.DataFrame({
                "rank": [rank],
                "index": ind,
                "importance": imp,
                "num_not_zero": n_pos,
                "prev_not_zero": n_prev,
                "mean_pos_label": mean_pos,
                "mean_neg_label": mean_neg,
                "concept_vocabulary": concept_vocabulary,
                "concept_domain": concept_domain,
                "concept_code": concept_code,
                "concept_name": concept_name,
                "quantile_bin": quantile_bin,
                "time_bin": time_bin,
            }),
        ))

    df_results = df_results.set_index("rank").sort_index()
    
    print(f"\n\n==============={task}=================")
    display(df_results)



===============long_los_sql=================


,index,importance,num_not_zero,prev_not_zero,mean_pos_label,mean_neg_label,concept_vocabulary,concept_domain,concept_code,concept_name,quantile_bin,time_bin
rank,,,,,,,,,,,,
1,0,2698,44055,1.000000,0.353331,-0.135551,NaN,NaN,NaN,Age,NaN,NaN
2,64813,534,24353,0.552786,5.682112,6.508040,NUCC,Visit,261Q00000X,Ambulatory Clinic / Center,NaN,36500 days
3,21926,475,6537,0.148383,0.244699,0.115295,SNOMED,Procedure,735414004,Insertion of catheter into peripheral vein,NaN,1 day
4,18994,354,11151,0.253115,0.021695,0.341897,SNOMED,Observation,405606005,Labour and delivery unit,NaN,1 day
5,23453,351,36048,0.818250,1.035530,0.951131,Visit,Visit,ERIP,Emergency Room and Inpatient Visit,NaN,1 day
6,10491,301,5011,0.113744,0.008187,0.154240,RxNorm,Drug,197806,ibuprofen 600 MG Oral Tablet,NaN,1 day
7,17893,288,1974,0.044808,0.088907,0.027889,SNOMED,Observation,309958005,Psychiatry department,NaN,1 day
8,9285,282,22479,0.510249,0.695702,0.439102,LOINC,Measurement,77147-7,Glomerular filtration rate/1.73 sq M.predicted...,NaN,1 day
9,11419,272,11552,0.262218,0.322964,0.238945,RxNorm,Drug,313782,acetaminophen 325 MG Oral Tablet,NaN,1 day




===============hyponatremia_lab_sql=================


,index,importance,num_not_zero,prev_not_zero,mean_pos_label,mean_neg_label,concept_vocabulary,concept_domain,concept_code,concept_name,quantile_bin,time_bin
rank,,,,,,,,,,,,
1,0,27083,43950,1.000000,0.506909,-0.004480,NaN,NaN,NaN,Age,NaN,NaN
2,17883,3338,967,0.022002,0.122078,0.021118,SNOMED,Observation,309993003,Surgical transplant department,NaN,1 day
3,79752,3260,13149,0.299181,1.272727,1.433031,LOINC,Measurement,2075-0,Chloride [Moles/volume] in Serum or Plasma,"104.0, 106.0",36500 days
4,9271,3074,22381,0.509238,0.693506,0.507609,LOINC,Measurement,77147-7,Glomerular filtration rate/1.73 sq M.predicted...,NaN,1 day
5,78814,3001,3648,0.083003,0.174026,0.082199,mimiciv_per_ethnicity,Race,UNKNOWN,UNKNOWN,NaN,36500 days
6,56517,2910,26682,0.607099,0.477922,0.608241,Gender,Gender,F,FEMALE,NaN,36500 days
7,9756,2206,11688,0.265939,0.389610,0.264846,RxNorm,Drug,1361615,"heparin sodium, porcine 5000 UNT/ML Injectable...",NaN,1 day
8,11933,2167,12725,0.289534,0.387013,0.288672,RxNorm,Drug,727820,10 ML sodium chloride 9 MG/ML Prefilled Syringe,NaN,1 day
9,9324,2083,3427,0.077975,0.103896,0.077746,Medicare Specialty,Visit,A0,Hospital,NaN,1 day




===============mortality_sql=================


...

,index,importance,num_not_zero,prev_not_zero,mean_pos_label,mean_neg_label,concept_vocabulary,concept_domain,concept_code,concept_name,quantile_bin,time_bin
rank,,,,,,,,,,,,
1,0,3265,43682,1.000000,0.463534,-0.033620,NaN,NaN,NaN,Age,NaN,NaN
2,64480,424,24139,0.552607,7.435003,6.146951,NUCC,Visit,261Q00000X,Ambulatory Clinic / Center,NaN,36500 days
3,26792,365,879,0.020123,0.186527,0.014044,LOINC,Measurement,718-7,Hemoglobin [Mass/volume] in Blood,"-inf, 7.800000190734863",1 day
4,56329,351,26513,0.606955,0.547393,0.611275,Gender,Gender,F,FEMALE,NaN,36500 days
5,21825,324,6349,0.145346,0.277590,0.138676,SNOMED,Procedure,735414004,Insertion of catheter into peripheral vein,NaN,1 day
6,11879,307,10176,0.232956,0.179418,0.236840,RxNorm,Drug,727633,sodium chloride Prefilled Syringe,NaN,1 day
7,11882,301,12608,0.288631,0.424171,0.278801,RxNorm,Drug,727820,10 ML sodium chloride 9 MG/ML Prefilled Syringe,NaN,1 day
8,23349,296,7805,0.178678,0.162830,0.181374,Visit,Visit,IP,Inpatient Visit,NaN,1 day
9,9898,281,3185,0.072913,0.149628,0.067349,RxNorm,Drug,1658717,"250 ML heparin sodium, porcine 100 UNT/ML Inje...",NaN,1 day




===============hypoglycemia_lab_sql=================


,index,importance,num_not_zero,prev_not_zero,mean_pos_label,mean_neg_label,concept_vocabulary,concept_domain,concept_code,concept_name,quantile_bin,time_bin
rank,,,,,,,,,,,,
1,0,25366,43983,1.000000,0.557166,-0.009259,NaN,NaN,NaN,Age,NaN,NaN
2,9284,3534,22411,0.509538,0.735744,0.505778,LOINC,Measurement,77147-7,Glomerular filtration rate/1.73 sq M.predicted...,NaN,1 day
3,64744,3146,23149,0.526317,6.012517,3.687939,LOINC,Measurement,77147-7,Glomerular filtration rate/1.73 sq M.predicted...,NaN,36500 days
4,23449,2669,35983,0.818112,1.023644,0.973327,Visit,Visit,ERIP,Emergency Room and Inpatient Visit,NaN,1 day
5,17894,2575,2438,0.055431,0.109875,0.054526,SNOMED,Observation,309971008,Cardiac surgery department,NaN,1 day
6,24604,2526,1589,0.036128,0.147427,0.035688,LOINC,Measurement,2345-7,Glucose [Mass/volume] in Serum or Plasma,"-inf, 83.0",1 day
7,64809,2361,27367,0.622218,0.631432,0.622065,Race,Observation,5,Sample collection duration,NaN,36500 days
8,24479,2094,2001,0.045495,0.168289,0.049672,LOINC,Measurement,2160-0,Creatinine [Mass/volume] in Serum or Plasma,"1.600000023841858, 2.4000000953674316",1 day
9,24480,2048,1958,0.044517,0.260083,0.052908,LOINC,Measurement,2160-0,Creatinine [Mass/volume] in Serum or Plasma,"2.4000000953674316, inf",1 day




===============thrombocytopenia_lab_sql=================


,index,importance,num_not_zero,prev_not_zero,mean_pos_label,mean_neg_label,concept_vocabulary,concept_domain,concept_code,concept_name,quantile_bin,time_bin
rank,,,,,,,,,,,,
1,0,28902,43912,1.000000,0.321071,-0.010122,NaN,NaN,NaN,Age,NaN,NaN
2,9265,4397,22352,0.509018,0.739940,0.501738,LOINC,Measurement,77147-7,Glomerular filtration rate/1.73 sq M.predicted...,NaN,1 day
3,78807,2930,3623,0.082506,0.135618,0.080832,mimiciv_per_ethnicity,Race,UNKNOWN,UNKNOWN,NaN,36500 days
4,64649,2900,23113,0.526348,8.251863,3.581865,LOINC,Measurement,77147-7,Glomerular filtration rate/1.73 sq M.predicted...,NaN,36500 days
5,9748,2733,11705,0.266556,0.254844,0.266925,RxNorm,Drug,1361615,"heparin sodium, porcine 5000 UNT/ML Injectable...",NaN,1 day
6,27102,2608,3491,0.079500,0.120715,0.089429,LOINC,Measurement,777-3,Platelets [#/volume] in Blood by Automated count,"115.0, 153.0",1 day
7,82484,2583,1487,0.033863,10.447840,0.376533,LOINC,Measurement,777-3,Platelets [#/volume] in Blood by Automated count,"-inf, 64.0",36500 days
8,56522,2460,26674,0.607442,0.429955,0.613037,Gender,Gender,F,FEMALE,NaN,36500 days
9,11929,2435,12711,0.289465,0.321162,0.288466,RxNorm,Drug,727820,10 ML sodium chloride 9 MG/ML Prefilled Syringe,NaN,1 day
